## VAR Model Order Selection

**Functions**

`tsa.VAR`

### Exercise 68
Using the same data as in the previous exercise, determine the optimal VAR order using:

1. AIC
2. HQIC
3. BIC
4. Likelihood-ratio testing using General-to-Specific

In [3]:
import pandas as pd
data = pd.read_hdf("./data/var-data.h5", "var_data")

In [7]:
import statsmodels.tsa.api as tsa
mod = tsa.VAR(data[["spread","gs1","deflg"]])
res = mod.fit(8, ic="bic")
res.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 27, Feb, 2020
Time:                     08:15:58
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                   -15.1574
Nobs:                     264.000    HQIC:                  -15.3276
Log likelihood:           935.526    FPE:                1.96653e-07
AIC:                     -15.4419    Det(Omega_mle):     1.81805e-07
--------------------------------------------------------------------
Results for equation spread
               coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------
const             0.112254         0.072096            1.557           0.119
L1.spread         1.068581         0.098461           10.853           0.000
L1.gs1           -0.051826         0.061315           -0.845           0.39

In [25]:
import numpy as np
from scipy import stats
ics = pd.DataFrame(columns=["AIC","HQIC","BIC", "LLF"],
                   index=np.arange(9), dtype=np.double)

for lag in range(9):
    res = mod.fit(lag, ic=None)
    ics.loc[lag, "AIC"] = res.aic
    ics.loc[lag, "HQIC"] = res.hqic
    ics.loc[lag, "BIC"] = res.bic
    ics.loc[lag, "LLF"] = res.llf
ics["LR"] = 2 * (ics.LLF - ics.LLF.shift(1))
ics["pval"] = 1 - stats.chi2(9).cdf(ics.LR)
# Pretty the values by truncating at 3 d.p.
ics.round(3)

C:\Anaconda\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Anaconda\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Anaconda\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,AIC,HQIC,BIC,LLF,LR,pval
0,-8.796,-8.779,-8.755,40.491,NaN,NaN
1,-15.249,-15.184,-15.087,904.460,1727.938,0.000
2,-15.442,-15.328,-15.157,935.526,62.133,0.000
3,-15.454,-15.291,-15.047,942.707,14.361,0.110
4,-15.507,-15.294,-14.976,955.196,24.979,0.003
5,-15.471,-15.207,-14.815,955.931,1.471,0.997
6,-15.553,-15.239,-14.772,972.085,32.306,0.000
7,-15.549,-15.185,-14.643,977.091,10.014,0.349
8,-15.578,-15.163,-14.545,986.309,18.435,0.030


In [22]:
ics[["AIC","HQIC","BIC"]].idxmin()


AIC     8
HQIC    2
BIC     2
dtype: int64